In [ ]:
# This notebook calculates the brain score and forcast score
# by Frank Hu, 2025-09

In [1]:
import os
import torch
import scipy.io
import numpy as np
import pandas as pd

from brain_score import time_series_ridge_cv_cuda

In [13]:
class CFG:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    design_matrix_path = '../Example_data/design_matrix_forcasting/design_mat_after_pca.mat'
    data_path = '../Example_data/hbo_data_forcast'
    result_path = '../Example_data/example_results_forcast/brain_score_results.mat'
    random_state = 42  # Random seed for reproducibility

In [ ]:
design_mat = scipy.io.loadmat(CFG.design_matrix_path)

(837, 20)

In [6]:
design_mat_full = [design_mat["pc_mat_1"], np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_2"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_3"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_4"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_5"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_6"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_7"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_8"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_9"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_10"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_11"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_12"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_13"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_14"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_15"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_16"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_17"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_18"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_19"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_20"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_21"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_22"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_23"]], axis=1), np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_24"]], axis=1),
                   np.concatenate([design_mat["pc_mat_1"], design_mat["pc_mat_25"]], axis=1),]


In [ ]:
# Get list of .mat files
files = [f for f in os.listdir(CFG.data_path) if f.endswith('.mat')]

num_files = len(files)
# num_channels = 20  # Each Y has 20 columns
num_channels = 32  # Each Y has 32 columns full brain
brain_score_Z_matrix = np.zeros((num_files, num_channels, len(design_mat_full)))  # Preallocate matrix for weights


(2, 32, (2, 32, 25))

In [ ]:
# Function to calculate brain scores
# This loop will take some time to run depending on the number of files and channels (e.g. two files with 32 channels took about 6 min with a NVIDIA RTX 3080 GPU)
for i, file_name in enumerate(files):
    # Load the .mat file
    file_path = os.path.join(CFG.data_path, file_name)
    data = scipy.io.loadmat(file_path)
    
    # Extract X and Y
    Y = data['hbodata'][0:837,:] # modified to fit the new size of the design matrix

    # Perform regression for each channel
    for j in range(num_channels):
        word_count = 0
        y_j = Y[:, j]  # Extract each channel
        for design_mat_cell in design_mat_full:
            X = design_mat_cell 
            results = time_series_ridge_cv_cuda(X, y_j, device=CFG.device)  # Least squares estimation
            brain_score_Z_matrix[i, j, word_count] = results['mean_z']  # Store weight
            word_count += 1
            
    print(f"{file_name} finished...")
    
    break   # <- stop after first file

# Now, W_matrix is (100, 20) containing all regression coefficients
print("Regression completed. W_matrix shape:", brain_score_Z_matrix.shape)

hbo_data_forcaststory_s02.mat finished...
Regression completed. W_matrix shape: (2, 32, 25)


In [14]:
# Save the results to a .mat file
scipy.io.savemat(CFG.result_path, {"brain_score": brain_score_Z_matrix})